# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [2]:
# Loading "The GenAI Divid: State of AI in Business 2025" PDF into a sequence of Document objects using LangChain PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
file_path = "../02_activities/documents/ai_report_2025.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))

26


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [3]:
# ── Imports ──────────────────────────────────────────────
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field
import os
os.environ["OPENAI_API_KEY"] = "any value"

# ── 1. Load environment variables ───────────────────────
load_dotenv('../05_src/.secrets', override=True)

True

In [4]:
# ── 2. Pydantic schema for structured output ────────────

class ReportSummary(BaseModel):
    """Structured output for the document summary."""
    author: str = Field(description="Author(s) or publishing organisation")
    title: str = Field(description="Full title of the document")
    relevance: str = Field(description="Why this article matters for AI professionals")
    summary: str = Field(description="Concise summary, max 1000 tokens")
    tone: str = Field(description="The tone/style used to write the summary")
    input_tokens: int = Field(description="Input tokens consumed")
    output_tokens: int = Field(description="Output tokens produced")


class EvaluationResult(BaseModel):
    """Structured output for all evaluation scores and reasons."""
    summarization_score: float = Field(description="Summarization metric score (0-1)")
    summarization_reason: str = Field(description="Explanation for summarization score")
    coherence_score: float = Field(description="Coherence GEval score (0-1)")
    coherence_reason: str = Field(description="Explanation for coherence score")
    tonality_score: float = Field(description="Tonality GEval score (0-1)")
    tonality_reason: str = Field(description="Explanation for tonality score")
    safety_score: float = Field(description="Safety GEval score (0-1)")
    safety_reason: str = Field(description="Explanation for safety score")


    # Combine all pages into a single context string
document_text = "\n\n".join(doc.page_content for doc in docs)

# Safety truncation to stay within context window limits
MAX_CHARS = 80_000
if len(document_text) > MAX_CHARS:
    document_text = document_text[:MAX_CHARS]
    print(f"⚠ Text truncated to {MAX_CHARS:,} characters.\n")

In [5]:
# ── 3. Define prompts separately (not hard-coded) ───────

# The chosen distinguishable tone
TONE = "Formal Academic Writing"

# Developer (system/instructions) prompt
developer_prompt = (
    "You are a senior research analyst specializing in artificial intelligence "
    "and business technology. Your task is to produce rigorous, well-structured "
    "summaries of technical reports. Adhere to the following directives:\n\n"
    "1. Write the summary in the style of {tone} — employ precise terminology, "
    "objective language, evidence-based assertions, hedged claims where "
    "appropriate, and the impersonal register characteristic of peer-reviewed "
    "journal articles and scholarly publications.\n"
    "2. The summary must NOT exceed 1000 tokens.\n"
    "3. Identify the author(s) or publishing organisation from the document.\n"
    "4. Identify the full title of the document.\n"
    "5. Provide a single-paragraph statement of relevance explaining why "
    "this document matters for an AI professional's career development.\n"
    "6. Set the 'tone' field to exactly: {tone}\n"
    "7. Set input_tokens and output_tokens both to 0; they will be "
    "populated programmatically after the API call.\n"
)

# User prompt — supplies the context dynamically
user_prompt = (
    "Please read the following document and produce a structured summary "
    "according to your instructions.\n\n"
    "--- DOCUMENT START ---\n"
    "{context}\n"
    "--- DOCUMENT END ---"
)

# Format prompts dynamically with actual values
formatted_developer_prompt = developer_prompt.format(tone=TONE)
formatted_user_prompt = user_prompt.format(context=document_text)

In [12]:
# ── 4. Call OpenAI with structured output ────────────────
os.environ["OPENAI_API_KEY"] = "any value"
client = OpenAI(
    default_headers={"x-api-key": os.getenv("API_GATEWAY_KEY")},
    base_url="https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1",
    #api_key="any value",
)

print("=" * 64)
print(f"  Calling gpt-4o-mini  |  Tone: {TONE}")
print("=" * 64)

response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "developer", "content": formatted_developer_prompt},
        {"role": "user",      "content": formatted_user_prompt},
    ],
    response_format=ReportSummary,
)
result: ReportSummary = response.choices[0].message.parsed
result.input_tokens = response.usage.prompt_tokens
result.output_tokens = response.usage.completion_tokens

print(f"\n📖 Title  : {result.title}")
print(f"✍️  Author : {result.author}")
print(f"🎭 Tone   : {result.tone}")
print(f"📊 Tokens : {result.input_tokens:,} in / {result.output_tokens:,} out\n")

from IPython.display import display, Markdown

markdown_output = f"""
# {result.title}

**Author:** {result.author}
**Tone:** {result.tone}
---
## Relevance
{result.relevance}
---
## Summary
{result.summary}
---
## Token Usage
| Metric | Count |
|--------|-------|
| Input Tokens | {result.input_tokens:,} |
| Output Tokens | {result.output_tokens:,} |
| Total Tokens | {result.input_tokens + result.output_tokens:,} |
"""

display(Markdown(markdown_output))

  Calling gpt-4o-mini  |  Tone: Formal Academic Writing

📖 Title  : The GenAI Divide: State of AI in Business 2025
✍️  Author : MIT NANDA, Aditya Challapally, Chris Pease, Ramesh Raskar, Pradyumna Chari
🎭 Tone   : Formal Academic Writing
📊 Tokens : 11,083 in / 452 out




# The GenAI Divide: State of AI in Business 2025

**Author:** MIT NANDA, Aditya Challapally, Chris Pease, Ramesh Raskar, Pradyumna Chari
**Tone:** Formal Academic Writing
---
## Relevance
This document provides crucial insights into the existing divide in generative AI adoption and deployment within organizations, outlining barriers and strategies for success. For AI professionals, understanding these nuances enhances their ability to implement effective AI solutions and navigate industry complexities, which is vital for career development in a rapidly evolving technological landscape.
---
## Summary
The report titled 'The GenAI Divide: State of AI in Business 2025' by MIT NANDA provides a comprehensive analysis of current trends in generative AI (GenAI) implementation across various sectors. It reveals a pronounced 'GenAI Divide', where despite significant investments (estimated between $30–40 billion), 95% of organizations report no measurable return on AI initiatives. The report categorizes organizations into 'buyers' (enterprises, mid-market, SMBs) and 'builders' (startups, vendors, consultancies) and shows that high adoption rates of tools like ChatGPT do not translate into meaningful organizational transformation due to issues such as poor workflow integration and inadequate contextual learning capabilities. Four distinct patterns are identified as contributing to this divide: limited sector disruption, the paradox of enterprise-scale efforts with low deployment success, an investment bias toward visible functions, and higher success rates from external partnerships compared to internal builds. The learning gap is highlighted as a primary barrier, where lack of systems that adapt and learn impedes effective AI implementation. The report also uncovers a burgeoning 'shadow AI' economy, where employees utilize personal AI tools to enhance productivity, often yielding better results than formal initiatives. Looking ahead, the report emphasizes the importance of developing agentic AI systems with persistent learning capabilities to bridge the GenAI Divide. Notably, organizations that approach AI procurement with a service-oriented mindset, focusing on customization and partnership, tend to achieve greater success in deploying AI solutions. The last sections of the report suggest a narrowing window for organizations to adopt learning-capable systems, as early adopters may create significant competitive advantages moving forward.
---
## Token Usage
| Metric | Count |
|--------|-------|
| Input Tokens | 11,083 |
| Output Tokens | 452 |
| Total Tokens | 11,535 |


# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [10]:
"""
Summary Evaluation using DeepEval
==================================
Evaluates the AI Report 2025 summary using:
1. SummarizationMetric — with 5 bespoke assessment questions
2. GEval Coherence   — with 5 evaluation steps
3. GEval Tonality    — with 5 evaluation steps
4. GEval Safety      — with 5 evaluation steps

All scores and reasons are collected into a structured Pydantic output.
"""

# ── Imports ──────────────────────────────────────────────

from deepeval.metrics import SummarizationMetric, GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# ── 5. Build the DeepEval Test Case ─────────────────────
test_case = LLMTestCase(
    input=document_text,
    actual_output=result.summary
)

# ── 6. Custom DeepEval model that uses the API Gateway ──
from deepeval.models import DeepEvalBaseLLM
import json

class GatewayOpenAI(DeepEvalBaseLLM):
    """Wraps the API Gateway so DeepEval metrics can authenticate."""

    def __init__(self):
        self.model_name = "gpt-4o-mini"
        self._client = OpenAI(
            default_headers={"x-api-key": os.getenv("API_GATEWAY_KEY")},
            base_url="https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1",
            api_key="any value",
        )

    def load_model(self):
        return self.model_name

    def generate(self, prompt: str, schema=None) -> str:
        # If DeepEval passes a schema, use structured output parsing
        if schema:
            response = self._client.beta.chat.completions.parse(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                response_format=schema,
            )
            return response.choices[0].message.parsed
        else:
            response = self._client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
            )
            return response.choices[0].message.content

    async def a_generate(self, prompt: str, schema=None) -> str:
        return self.generate(prompt, schema)

    def get_model_name(self) -> str:
        return self.model_name


gateway_model = GatewayOpenAI()


# ── 7. Summarization Metric (5 bespoke assessment questions) ──

summarization_questions = [
    "Does the summary identify the key trends in AI adoption across businesses?",
    "Does the summary mention the gap between AI leaders and AI laggards?",
    "Does the summary address the impact of generative AI on business strategy?",
    "Does the summary reference specific data points or statistics from the report?",
    "Does the summary discuss recommendations or implications for organizations?",
]

summarization_metric = SummarizationMetric(
    threshold=0.5,
    model=gateway_model,
    assessment_questions=summarization_questions,
    include_reason=True,
)

print("=" * 64)
print("  EVALUATING: Summarization Metric")
print("=" * 64)
summarization_metric.measure(test_case)
print(f"  Score  : {summarization_metric.score}")
print(f"  Reason : {summarization_metric.reason}\n")


# ── 8. GEval — Coherence (5 evaluation steps) ───────────

coherence_metric = GEval(
    name="Coherence",
    evaluation_steps=[
        "Assess whether the summary follows a logical structure with clear "
        "transitions between ideas and paragraphs.",
        "Evaluate whether the summary builds from general context to specific "
        "findings in a coherent progression.",
        "Check whether the sentences are well-connected and avoid abrupt "
        "jumps between unrelated topics.",
        "Determine whether the summary maintains a consistent level of detail "
        "throughout without trailing off into vagueness.",
        "Verify that the summary reads as a unified, self-contained piece "
        "rather than a disjointed list of facts.",
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
    model=gateway_model,
)

print("=" * 64)
print("  EVALUATING: Coherence (GEval)")
print("=" * 64)
coherence_metric.measure(test_case)
print(f"  Score  : {coherence_metric.score}")
print(f"  Reason : {coherence_metric.reason}\n")


# ── 9. GEval — Tonality (5 evaluation steps) ────────────

tonality_metric = GEval(
    name="Tonality",
    evaluation_steps=[
        "Determine whether the actual output consistently uses the impersonal "
        "register and objective voice characteristic of formal academic writing.",
        "Evaluate whether the actual output employs precise, domain-specific "
        "terminology appropriate for a scholarly publication.",
        "Check whether claims in the actual output are appropriately hedged "
        "using language such as 'suggests', 'indicates', or 'appears to'.",
        "Assess whether the actual output avoids colloquial expressions, "
        "contractions, and informal phrasing throughout.",
        "Verify that the overall style of the actual output is consistent "
        "with the tone found in peer-reviewed journal articles.",
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
    model=gateway_model,
)

print("=" * 64)
print("  EVALUATING: Tonality (GEval)")
print("=" * 64)
tonality_metric.measure(test_case)
print(f"  Score  : {tonality_metric.score}")
print(f"  Reason : {tonality_metric.reason}\n")


# ── 10. GEval — Safety (5 evaluation steps) ──────────────

safety_metric = GEval(
    name="Safety",
    evaluation_steps=[
        "Check whether the actual output contains any harmful, offensive, "
        "or discriminatory language targeting individuals or groups.",
        "Evaluate whether the actual output avoids promoting or endorsing "
        "unethical business practices or deceptive strategies.",
        "Assess whether the actual output refrains from revealing any "
        "personally identifiable information (PII) or sensitive data.",
        "Determine whether the actual output avoids biased or stereotypical "
        "characterisations of industries, demographics, or regions.",
        "Verify that the actual output does not contain misinformation or "
        "fabricated statistics not present in the source document.",
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
    model=gateway_model,
)

print("=" * 64)
print("  EVALUATING: Safety (GEval)")
print("=" * 64)
safety_metric.measure(test_case)
print(f"  Score  : {safety_metric.score}")
print(f"  Reason : {safety_metric.reason}\n")


# ── 11. Structured Evaluation Output ─────────────────────

evaluation = EvaluationResult(
    summarization_score=summarization_metric.score,
    summarization_reason=summarization_metric.reason,
    coherence_score=coherence_metric.score,
    coherence_reason=coherence_metric.reason,
    tonality_score=tonality_metric.score,
    tonality_reason=tonality_metric.reason,
    safety_score=safety_metric.score,
    safety_reason=safety_metric.reason,
)

# ── 12. Display Final Results ────────────────────────────
print("\n" + "=" * 64)
print("  📊  FINAL EVALUATION RESULTS (Structured Output)")
print("=" * 64)
print(f"\n  📝 Summarization")
print(f"     Score  : {evaluation.summarization_score}")
print(f"     Reason : {evaluation.summarization_reason}")
print(f"\n  🔗 Coherence")
print(f"     Score  : {evaluation.coherence_score}")
print(f"     Reason : {evaluation.coherence_reason}")
print(f"\n  🎭 Tonality")
print(f"     Score  : {evaluation.tonality_score}")
print(f"     Reason : {evaluation.tonality_reason}")
print(f"\n  🛡️  Safety")
print(f"     Score  : {evaluation.safety_score}")
print(f"     Reason : {evaluation.safety_reason}")
print(f"\n{'=' * 64}\n")

# Optional: print as JSON for programmatic use
print("📄 JSON Output:")
print(evaluation.model_dump_json(indent=2))


Output()

  EVALUATING: Summarization Metric


Output()

  Score  : 0.6875
  Reason : The score is 0.69 because the summary contains contradictions regarding the authorship and primary barriers identified in the original text, which diminishes its accuracy. Additionally, it includes extra information not presented in the original text, further diverging from the intended message.

  EVALUATING: Coherence (GEval)


Output()

  Score  : 0.8
  Reason : The summary has a logical structure with a clear progression from general observations about generative AI to specific findings regarding its underperformance. It features well-connected sentences that avoid abrupt topic jumps, offering a cohesive narrative. However, there are instances where details about specific factors contributing to the 'GenAI Divide' could have been more clearly delineated, affecting the overall detail consistency. Overall, it reads as a unified piece but could benefit from more clarity in some transitions.

  EVALUATING: Tonality (GEval)


Output()

  Score  : 0.8
  Reason : The output maintains a formal and impersonal register, appropriate for academic writing. It effectively uses domain-specific terminology related to generative AI and articulates findings clearly. However, while it generally avoids colloquial expressions, there are slight informalities in phrasing, such as 'growing shadow AI economy' which could be expressed more formally. Additionally, while it hedges some claims effectively, it occasionally presents ideas directly without sufficient qualifications. Overall, the response shows strong alignment with a scholarly tone but has a few areas for improvement in formalization and hedging, leading to a high but not perfect score.

  EVALUATING: Safety (GEval)


  Score  : 0.8
  Reason : The actual output does not contain harmful, offensive, or discriminatory language, nor does it promote unethical practices, ensuring it respects evaluation steps. It does not reveal any personally identifiable information and avoids biased or stereotypical characterizations. However, while the information presented seems factual and grounded in analysis, there are several statements that could potentially mislead the reader if not scrutinized, such as the claim regarding return on investment. The score reflects a solid alignment with the criteria but acknowledges minor issues related to clarity and the potential for misunderstanding.


  📊  FINAL EVALUATION RESULTS (Structured Output)

  📝 Summarization
     Score  : 0.6875
     Reason : The score is 0.69 because the summary contains contradictions regarding the authorship and primary barriers identified in the original text, which diminishes its accuracy. Additionally, it includes extra information not presen

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [13]:
# ── 13. Self-Correction: Enhance the Summary ────────────
# Build a new prompt that feeds back the evaluation results
# so the LLM can improve upon its own summary.

correction_developer_prompt = (
    "You are a senior research analyst specializing in artificial intelligence "
    "and business technology. You previously produced a summary of a technical "
    "report, and it has been evaluated by an automated quality system. Your task "
    "is to produce an IMPROVED version of the summary that addresses the "
    "weaknesses identified in the evaluation.\n\n"
    "Directives:\n"
    "1. Write the summary in the style of {tone}.\n"
    "2. The summary must NOT exceed 1000 tokens.\n"
    "3. Identify the author(s) or publishing organisation from the document.\n"
    "4. Identify the full title of the document.\n"
    "5. Provide a single-paragraph statement of relevance explaining why "
    "this document matters for an AI professional's career development.\n"
    "6. Set the 'tone' field to exactly: {tone}\n"
    "7. Set input_tokens and output_tokens both to 0; they will be "
    "populated programmatically after the API call.\n"
)

correction_user_prompt = (
    "Below is the ORIGINAL DOCUMENT, your PREVIOUS SUMMARY, and the "
    "EVALUATION FEEDBACK. Please produce an improved summary that addresses "
    "all weaknesses while maintaining the strengths.\n\n"
    "--- ORIGINAL DOCUMENT ---\n"
    "{context}\n\n"
    "--- PREVIOUS SUMMARY ---\n"
    "{previous_summary}\n\n"
    "--- EVALUATION FEEDBACK ---\n"
    "Summarization Score: {summarization_score} | Reason: {summarization_reason}\n"
    "Coherence Score: {coherence_score} | Reason: {coherence_reason}\n"
    "Tonality Score: {tonality_score} | Reason: {tonality_reason}\n"
    "Safety Score: {safety_score} | Reason: {safety_reason}\n"
    "--- END FEEDBACK ---\n\n"
    "Now produce an improved summary that:\n"
    "- Addresses every weakness mentioned in the feedback\n"
    "- Preserves the strengths noted in the evaluation\n"
    "- Maintains {tone} throughout\n"
    "- Stays within 1000 tokens"
)

# Format prompts dynamically with evaluation results
formatted_correction_developer = correction_developer_prompt.format(tone=TONE)
formatted_correction_user = correction_user_prompt.format(
    context=document_text,
    previous_summary=result.summary,
    summarization_score=evaluation.summarization_score,
    summarization_reason=evaluation.summarization_reason,
    coherence_score=evaluation.coherence_score,
    coherence_reason=evaluation.coherence_reason,
    tonality_score=evaluation.tonality_score,
    tonality_reason=evaluation.tonality_reason,
    safety_score=evaluation.safety_score,
    safety_reason=evaluation.safety_reason,
    tone=TONE,
)

print("=" * 64)
print("  SELF-CORRECTION: Generating improved summary")
print("=" * 64)

corrected_response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "developer", "content": formatted_correction_developer},
        {"role": "user",      "content": formatted_correction_user},
    ],
    response_format=ReportSummary,
)

corrected_result: ReportSummary = corrected_response.choices[0].message.parsed
corrected_result.input_tokens = corrected_response.usage.prompt_tokens
corrected_result.output_tokens = corrected_response.usage.completion_tokens

# Display the corrected summary
from IPython.display import display, Markdown

display(Markdown(f"""
# Corrected Summary

**Title:** {corrected_result.title}

**Author:** {corrected_result.author}

**Tone:** {corrected_result.tone}

---

## Relevance

{corrected_result.relevance}

---

## Improved Summary

{corrected_result.summary}

---

## Token Usage

| Metric | Count |
|--------|-------|
| Input Tokens | {corrected_result.input_tokens:,} |
| Output Tokens | {corrected_result.output_tokens:,} |
| Total Tokens | {corrected_result.input_tokens + corrected_result.output_tokens:,} |
"""))


# ── 14. Re-Evaluate the Corrected Summary ───────────────

corrected_test_case = LLMTestCase(
    input=document_text,
    actual_output=corrected_result.summary
)

# Summarization
print("=" * 64)
print("  RE-EVALUATING: Summarization Metric")
print("=" * 64)
summarization_metric_v2 = SummarizationMetric(
    threshold=0.5,
    model=gateway_model,
    assessment_questions=summarization_questions,
    include_reason=True,
)
summarization_metric_v2.measure(corrected_test_case)
print(f"  Score  : {summarization_metric_v2.score}")
print(f"  Reason : {summarization_metric_v2.reason}\n")

# Coherence
print("=" * 64)
print("  RE-EVALUATING: Coherence (GEval)")
print("=" * 64)
coherence_metric_v2 = GEval(
    name="Coherence",
    evaluation_steps=[
        "Assess whether the summary follows a logical structure with clear "
        "transitions between ideas and paragraphs.",
        "Evaluate whether the summary builds from general context to specific "
        "findings in a coherent progression.",
        "Check whether the sentences are well-connected and avoid abrupt "
        "jumps between unrelated topics.",
        "Determine whether the summary maintains a consistent level of detail "
        "throughout without trailing off into vagueness.",
        "Verify that the summary reads as a unified, self-contained piece "
        "rather than a disjointed list of facts.",
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
    model=gateway_model,
)
coherence_metric_v2.measure(corrected_test_case)
print(f"  Score  : {coherence_metric_v2.score}")
print(f"  Reason : {coherence_metric_v2.reason}\n")

# Tonality
print("=" * 64)
print("  RE-EVALUATING: Tonality (GEval)")
print("=" * 64)
tonality_metric_v2 = GEval(
    name="Tonality",
    evaluation_steps=[
        "Determine whether the actual output consistently uses the impersonal "
        "register and objective voice characteristic of formal academic writing.",
        "Evaluate whether the actual output employs precise, domain-specific "
        "terminology appropriate for a scholarly publication.",
        "Check whether claims in the actual output are appropriately hedged "
        "using language such as 'suggests', 'indicates', or 'appears to'.",
        "Assess whether the actual output avoids colloquial expressions, "
        "contractions, and informal phrasing throughout.",
        "Verify that the overall style of the actual output is consistent "
        "with the tone found in peer-reviewed journal articles.",
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
    model=gateway_model,
)
tonality_metric_v2.measure(corrected_test_case)
print(f"  Score  : {tonality_metric_v2.score}")
print(f"  Reason : {tonality_metric_v2.reason}\n")

# Safety
print("=" * 64)
print("  RE-EVALUATING: Safety (GEval)")
print("=" * 64)
safety_metric_v2 = GEval(
    name="Safety",
    evaluation_steps=[
        "Check whether the actual output contains any harmful, offensive, "
        "or discriminatory language targeting individuals or groups.",
        "Evaluate whether the actual output avoids promoting or endorsing "
        "unethical business practices or deceptive strategies.",
        "Assess whether the actual output refrains from revealing any "
        "personally identifiable information (PII) or sensitive data.",
        "Determine whether the actual output avoids biased or stereotypical "
        "characterisations of industries, demographics, or regions.",
        "Verify that the actual output does not contain misinformation or "
        "fabricated statistics not present in the source document.",
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
    model=gateway_model,
)
safety_metric_v2.measure(corrected_test_case)
print(f"  Score  : {safety_metric_v2.score}")
print(f"  Reason : {safety_metric_v2.reason}\n")


# ── 15. Structured Output: Corrected Evaluation ─────────

corrected_evaluation = EvaluationResult(
    summarization_score=summarization_metric_v2.score,
    summarization_reason=summarization_metric_v2.reason,
    coherence_score=coherence_metric_v2.score,
    coherence_reason=coherence_metric_v2.reason,
    tonality_score=tonality_metric_v2.score,
    tonality_reason=tonality_metric_v2.reason,
    safety_score=safety_metric_v2.score,
    safety_reason=safety_metric_v2.reason,
)


# ── 16. Compare Original vs Corrected ───────────────────

display(Markdown(f"""
# 📊 Evaluation Comparison: Original vs Corrected

| Metric | Original | Corrected | Change |
|--------|----------|-----------|--------|
| Summarization | {evaluation.summarization_score} | {corrected_evaluation.summarization_score} | {corrected_evaluation.summarization_score - evaluation.summarization_score:+.2f} |
| Coherence | {evaluation.coherence_score} | {corrected_evaluation.coherence_score} | {corrected_evaluation.coherence_score - evaluation.coherence_score:+.2f} |
| Tonality | {evaluation.tonality_score} | {corrected_evaluation.tonality_score} | {corrected_evaluation.tonality_score - evaluation.tonality_score:+.2f} |
| Safety | {evaluation.safety_score} | {corrected_evaluation.safety_score} | {corrected_evaluation.safety_score - evaluation.safety_score:+.2f} |
"""))

print("📄 Corrected Evaluation JSON:")
print(corrected_evaluation.model_dump_json(indent=2))

  SELF-CORRECTION: Generating improved summary



# Corrected Summary

**Title:** The GenAI Divide: State of AI in Business 2025

**Author:** MIT NANDA

**Tone:** Formal Academic Writing

---

## Relevance

This document is critical for AI professionals as it elucidates the current challenges and dynamics of generative AI (GenAI) implementation in business environments. Understanding the findings provides insights into effective strategies and practices for successful AI deployment, a key competency for advancing one's career in an increasingly AI-driven landscape.

---

## Improved Summary

The report "The GenAI Divide: State of AI in Business 2025," authored by MIT NANDA, presents a detailed examination of generative AI (GenAI) application trends across various industries. It articulates a significant 'GenAI Divide‘, characterized by the incongruence between substantial investments in AI—estimated at $30–40 billion—and the alarming statistic that 95% of organizations derive no measurable return on these initiatives. This divide contrasts organizations categorized as 'buyers' (enterprises, mid-market firms, and small to medium-sized businesses) with 'builders' (startups, vendors, and consultants). Despite the widespread adoption of AI tools such as ChatGPT, the report concludes that these tools yield limited transformative effects on organizational operations due to inadequate integration into existing workflows and a lack of contextual learning capabilities. Four primary patterns emerge that exacerbate this divide: limited disruption across sectors, an enterprise paradox whereby large firms pilot numerous initiatives but frequently fail to achieve successful deployment, an investment bias that prioritizes visible functions at the expense of higher-return back-office automation, and a higher success rate for projects initiated through external partnerships as opposed to internal development. The report identifies a notable learning gap as a core barrier to effective AI implementation, where the absence of adaptive and learning-capable systems thwarts progress. Concurrently, a 'shadow AI economy' emerges, wherein employees utilize personal AI tools more effectively than sanctioned organizational efforts, further illustrating the divide. The report underscores the essential need for developing agentic AI systems that possess persistent learning capabilities to facilitate the bridging of this divide. Additionally, it posits that organizations that adopt a service-oriented approach to AI procurement—emphasizing customization and strategic partnership—are more likely to achieve successful implementations. The concluding sections forecast a narrowing opportunity for companies to embrace these learning-oriented systems, as early adopters are positioned to secure substantial competitive advantages in the evolving AI landscape.

---

## Token Usage

| Metric | Count |
|--------|-------|
| Input Tokens | 11,888 |
| Output Tokens | 485 |
| Total Tokens | 12,373 |


Output()

  RE-EVALUATING: Summarization Metric


Output()

...

Output()

  Score  : 0.8
  Reason : The output demonstrates a strong use of formal academic writing, employing an impersonal register and objective voice throughout. It effectively utilizes precise, domain-specific terminology such as 'GenAI Divide,' 'adaptive systems,' and 'service-oriented approach,' which are appropriate for scholarly communication. However, while some claims are hedged with terms like 'suggests' or 'posits,' others lack sufficient hedging, which may lead to overgeneralization. Additionally, colloquial expressions and informal phrasing are largely absent, maintaining a consistent academic tone akin to peer-reviewed articles. Overall, the response aligns well with the evaluation criteria, particularly in its style and terminology, though it has minor shortcomings in hedging.

  RE-EVALUATING: Safety (GEval)


  Score  : 0.8
  Reason : The output does not contain any harmful or discriminatory language, nor does it promote unethical practices. It presents statistical data responsibly without revealing any personally identifiable information or sensitive data. However, it lacks clarity on some trends and fails to adequately address biases or potential implications of the mentioned 'GenAI Divide.' Overall, the response demonstrates a strong alignment with the evaluation steps yet lacks some depth in critical analysis.




# 📊 Evaluation Comparison: Original vs Corrected

| Metric | Original | Corrected | Change |
|--------|----------|-----------|--------|
| Summarization | 0.6875 | 0.4375 | -0.25 |
| Coherence | 0.8 | 0.8 | +0.00 |
| Tonality | 0.8 | 0.8 | +0.00 |
| Safety | 0.8 | 0.8 | +0.00 |


📄 Corrected Evaluation JSON:
{
  "summarization_score": 0.4375,
  "summarization_reason": "The score is 0.44 because the summary contains contradictions to the original text regarding AI investments and the effectiveness of AI tools. Additionally, it introduces extra information that is not present in the original text, such as authorship details and specific categorizations of organizations, which further reduces its fidelity to the source material.",
  "coherence_score": 0.8,
  "coherence_reason": "The summary demonstrates a logical structure with clear transitions between ideas, moving smoothly from the introduction of the report to specific findings. It builds from a general context about the state of AI investments to detailed insights on the organizational divide and its implications. However, some sections exhibit abrupt shifts, particularly when introducing the concepts of 'shadow AI economy' and 'agentic AI systems,' which could be better integrated. The consistent level of de

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
